In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
import numpy as np

In [2]:
# 1. Muat Data
df = pd.read_csv('StudentPerformanceFactors.csv')

In [3]:
# --- Pemeriksaan Awal dan Pembersihan Data ---
print("Shape data awal:", df.shape)
# Menghapus baris dengan nilai yang hilang (jika ada, untuk tujuan demonstrasi)
# Dalam skenario nyata, kita bisa melakukan imputasi (mengisi nilai hilang)
df.dropna(inplace=True)
print("Shape data setelah dropna:", df.shape)

Shape data awal: (6607, 20)
Shape data setelah dropna: (6378, 20)


In [4]:
# --- 2. Definisi Variabel dan Tipe Data ---

# Variabel Target
TARGET_COL = 'Exam_Score'

# Variabel Numerik (Continuous)
NUMERIC_COLS = ['Hours_Studied', 'Attendance', 'Sleep_Hours', 'Previous_Scores', 'Tutoring_Sessions', 'Physical_Activity']

# Variabel Kategorikal Ordinal (memiliki urutan/tingkatan)
ORDINAL_COLS = ['Parental_Involvement', 'Access_to_Resources', 'Motivation_Level', 'Family_Income', 'Teacher_Quality', 'Parental_Education_Level', 'Distance_from_Home']

# Variabel Kategorikal Nominal (tidak memiliki urutan)
NOMINAL_COLS = ['Extracurricular_Activities', 'Internet_Access', 'School_Type', 'Peer_Influence', 'Learning_Disabilities', 'Gender']

In [6]:
# --- 3. Definisi Pemetaan Ordinal ---
# Mendefinisikan urutan untuk setiap fitur Ordinal
# Catatan: Urutan ini penting untuk mempertahankan hierarki informasi
ordinal_categories = {
    'Parental_Involvement': ['Low', 'Medium', 'High'],
    'Access_to_Resources': ['Low', 'Medium', 'High'],
    'Motivation_Level': ['Low', 'Medium', 'High'],
    'Family_Income': ['Low', 'Medium', 'High'],
    'Teacher_Quality': ['Low', 'Medium', 'High'],
    'Parental_Education_Level': ['High School', 'College', 'Postgraduate'],
    'Distance_from_Home': ['Near', 'Moderate', 'Far'] # Asumsi urutan jarak
}

# Menyusun list urutan kategori berdasarkan dictionary
categories_list = [ordinal_categories[col] for col in ORDINAL_COLS]

In [7]:
# --- 4. Feature Engineering Pipeline (ColumnTransformer) ---
# Preprocessor untuk fitur numerik: Scaling
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])
# Preprocessor untuk fitur ordinal: Ordinal Encoding
ordinal_transformer = Pipeline(steps=[
    ('ordinal_encoder', OrdinalEncoder(categories=categories_list, handle_unknown='error'))
])
# Preprocessor untuk fitur nominal: One-Hot Encoding
nominal_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
# Menggabungkan semua transformer menggunakan ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, NUMERIC_COLS),
        ('ord', ordinal_transformer, ORDINAL_COLS),
        ('nom', nominal_transformer, NOMINAL_COLS)
    ],
    remainder='passthrough' # Biarkan kolom lain (seperti target) tetap
)

In [8]:
# --- 5. Penerapan Transformasi ---
# Memisahkan fitur (X) dan target (y)
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]
# Menerapkan ColumnTransformer pada data fitur (X)
X_processed_array = preprocessor.fit_transform(X)
# Mendapatkan nama fitur baru setelah transformasi
feature_names = (
    NUMERIC_COLS + 
    ORDINAL_COLS + 
    list(preprocessor.named_transformers_['nom']['onehot'].get_feature_names_out(NOMINAL_COLS))
)

In [9]:
# Membuat DataFrame baru untuk X_processed
X_processed = pd.DataFrame(X_processed_array, columns=feature_names)

# --- 6. Contoh Feature Creation (Interaksi Fitur) ---
# Membuat fitur interaksi sederhana: Kualitas Guru dikalikan dengan Keterlibatan Orang Tua
# Fitur ordinal sudah menjadi numerik setelah OrdinalEncoder, sehingga bisa dikalikan
X_processed['Teacher_x_Parental_Inv'] = (
    X_processed['Parental_Involvement'] * X_processed['Teacher_Quality']
)

In [10]:
# --- 7. Hasil Akhir ---
print("\n" + "="*50)
print("HASIL FEATURE ENGINEERING")
print("="*50)
print("\nDataFrame Fitur yang telah diproses (X_processed):")
print(X_processed.head())
print("\nShape DataFrame Fitur akhir:", X_processed.shape)
print("\nInformasi Tipe Data Final:")
print(X_processed.info(verbose=False))


HASIL FEATURE ENGINEERING

DataFrame Fitur yang telah diproses (X_processed):
   Hours_Studied  Attendance  Sleep_Hours  Previous_Scores  Tutoring_Sessions  \
0       0.505079    0.344520    -0.023819        -0.143491          -1.211858   
1      -0.163260   -1.387109     0.657418        -1.115763           0.409036   
2       0.672163    1.556661    -0.023819         1.106573           0.409036   
3       1.507587    0.777428     0.657418         1.592709          -0.401411   
4      -0.163260    1.037172    -0.705056        -0.699075           1.219483   

   Physical_Activity  Parental_Involvement  Access_to_Resources  \
0           0.026516                   0.0                  2.0   
1           0.998480                   0.0                  1.0   
2           0.998480                   1.0                  1.0   
3           0.998480                   0.0                  1.0   
4           0.998480                   1.0                  1.0   

   Motivation_Level  Family_Inc

In [11]:
# --- 8. Menggabungkan Fitur dan Target (Pilihan) ---
# Untuk kemudahan, kita gabungkan X_processed dan y menjadi satu DataFrame
# Kita harus mengatur ulang indeks y agar sesuai dengan indeks X_processed
y.index = X_processed.index
df_processed_final = pd.concat([X_processed, y], axis=1)

In [12]:
# --- 9. Menyimpan Hasil ke File CSV ---
# Definisikan nama file output
OUTPUT_FILE_NAME = 'StudentPerformance_FE_Processed.csv'
# Simpan DataFrame yang telah diproses ke file CSV
df_processed_final.to_csv(OUTPUT_FILE_NAME, index=False)
print("\n" + "="*50)
print("PENYIMPANAN HASIL FEATURE ENGINEERING")
print("="*50)
print(f"Dataframe yang telah diproses berhasil disimpan sebagai: **{OUTPUT_FILE_NAME}**")
print(f"Jumlah baris dan kolom data yang disimpan: {df_processed_final.shape}")
print(f"Lima baris pertama dari data yang disimpan:\n")
print(df_processed_final.head())


PENYIMPANAN HASIL FEATURE ENGINEERING
Dataframe yang telah diproses berhasil disimpan sebagai: **StudentPerformance_FE_Processed.csv**
Jumlah baris dan kolom data yang disimpan: (6378, 28)
Lima baris pertama dari data yang disimpan:

   Hours_Studied  Attendance  Sleep_Hours  Previous_Scores  Tutoring_Sessions  \
0       0.505079    0.344520    -0.023819        -0.143491          -1.211858   
1      -0.163260   -1.387109     0.657418        -1.115763           0.409036   
2       0.672163    1.556661    -0.023819         1.106573           0.409036   
3       1.507587    0.777428     0.657418         1.592709          -0.401411   
4      -0.163260    1.037172    -0.705056        -0.699075           1.219483   

   Physical_Activity  Parental_Involvement  Access_to_Resources  \
0           0.026516                   0.0                  2.0   
1           0.998480                   0.0                  1.0   
2           0.998480                   1.0                  1.0   
3         